In [ ]:
## Logistic Regression
# Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import os
import joblib

# Get current working directory
data_path = os.getcwd()

# Specify file paths based on current directory
image1_path = os.path.join(data_path, "image1.txt")
image2_path = os.path.join(data_path, "image2.txt")
image3_path = os.path.join(data_path, "image3.txt")

# Load the data
image_1 = pd.read_csv(image1_path, delim_whitespace=True, header=None)
image_2 = pd.read_csv(image2_path, delim_whitespace=True, header=None)
image_3 = pd.read_csv(image3_path, delim_whitespace=True, header=None)

# Define column names
column_names = ['y_coor', 'x_coor', 'expert_label', 'NDAI', 'SD', 'CORR',
                'Radiance_angle_DF', 'Radiance_angle_CF', 'Radiance_angle_BF',
                'Radiance_angle_AF', 'Radiance_angle_AN']

# Assign column names to the datasets
image_1.columns = column_names
image_2.columns = column_names
image_3.columns = column_names

# Filter out uncertain data (expert_label = 0)
train_on1 = image_1[image_1['expert_label'] != 0]
validate_on2 = image_2[image_2['expert_label'] != 0]

# Select features (drop expert_label and coordinates)
x_train1 = train_on1.drop(columns=['expert_label', 'x_coor', 'y_coor'])
y_train1 = train_on1['expert_label']

x_validate2 = validate_on2.drop(columns=['expert_label', 'x_coor', 'y_coor'])
y_validate2 = validate_on2['expert_label']

# Combine training and validation data for cross-validation
X = pd.concat([x_train1, x_validate2], ignore_index=True)
y = pd.concat([y_train1, y_validate2], ignore_index=True)

# Create test fold array for cross-validation
test_fold = np.ones(len(X))
test_fold[:len(x_train1)] = 0  # Image 1 as training (0)
test_fold[len(x_train1):] = 1  # Image 2 as validation (1)

# Create PredefinedSplit object
ps = PredefinedSplit(test_fold)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define logistic regression model and hyperparameters for tuning
log_reg = LogisticRegression(max_iter=500, random_state=42)
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],        # Regularization strength
    'penalty': ['l1', 'l2', 'elasticnet'],  # Regularization type
    'solver': ['saga'],                    # Solver supporting elasticnet
    'l1_ratio': [0, 0.5, 1]               # Only used for elasticnet penalty
}

# Define multiple scoring metrics
scoring = {
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy'
}

# Use GridSearchCV for logistic regression
grid_search = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid,
    scoring=scoring,
    cv=ps,
    refit='roc_auc',
    n_jobs=-1
)

# Fit the model with cross-validation
grid_search.fit(X_scaled, y)

# Access and print the best parameters and scores
best_params = grid_search.best_params_
best_roc_auc = grid_search.cv_results_['mean_test_roc_auc'].max()
best_accuracy = grid_search.cv_results_['mean_test_accuracy'].max()

print("Best ROC AUC:", best_roc_auc)
print("Best Accuracy:", best_accuracy)
print("Best Parameters:", best_params)

# Save the best model
best_log_reg = grid_search.best_estimator_
joblib.dump(best_log_reg, 'best_logistic_regression_model.pkl')

# Evaluate on test set (image 3)
test_on3 = image_3[image_3['expert_label'] != 0]
X_test = test_on3.drop(columns=['expert_label', 'x_coor', 'y_coor'])
y_test = test_on3['expert_label']
X_test_scaled = scaler.transform(X_test)

# Predict and evaluate on test data
y_test_pred = best_log_reg.predict(X_test_scaled)

print("Test set classification report:")
print(classification_report(y_test, y_test_pred))

print("Test set confusion matrix:")
print(confusion_matrix(y_test, y_test_pred))

/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when pena